In [2]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'g:\2025 - Ashoka\Ashoka25_Env\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, transforms
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import timm
import numpy as np
import random
import copy

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Set random seed
sit_number = 123456
torch.manual_seed(sit_number)
np.random.seed(sit_number)
random.seed(sit_number)

In [5]:
# 1. Transformasi dan augmentasi data dengan random_state dan modifikasi augmentasi
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomAffine(degrees=20, scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
# 2. Path dataset dan DataLoader
train_dir = "NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/train"
val_dir = "NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/val"
test_dir = "NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test"

train_data = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_data = datasets.ImageFolder(root=val_dir, transform=val_test_transform)
test_data = datasets.ImageFolder(root=test_dir, transform=val_test_transform)

# Menggunakan resampling untuk menangani ketidakseimbangan kelas
targets = [label for _, label in train_data]
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(targets), y=targets)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# Dataloader
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Output dataset counts
print(f'Train dataset size: {len(train_data)}')
print(f'Validation dataset size: {len(val_data)}')
print(f'Test dataset size: {len(test_data)}')

Train dataset size: 122
Validation dataset size: 40
Test dataset size: 43


# ConvnextV2

In [21]:
# 3. model ConvNeXt V2
model = timm.create_model('convnextv2_base', pretrained=True)

# Adaptive Pooling untuk memastikan output dari convolutional layer selalu memiliki ukuran tetap
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

In [22]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.head.parameters(), 'lr': 0.0001},
    {'params': model.stem.parameters(), 'lr': 0.00001},
    {'params': model.stages.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

In [23]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [24]:
# 6. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)

    # Initialize history lists to store metrics
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Append train loss and accuracy to history lists
        train_loss_history.append(running_loss / len(train_loader))
        # You need to calculate and append train accuracy here if needed

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        # Append val loss and accuracy to history lists
        val_loss_history.append(val_loss / len(val_loader))
        val_accuracy_history.append(accuracy)

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/ConvnextV2_samping.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history

In [25]:
# 7. Evaluasi Model pada Data Uji
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
    specificity = tn / (tn + fp)

    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

In [26]:
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [27]:
# 8. Train model and save the best weights
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 1.1843
Accuracy: 0.6000, Precision: 0.6667, Recall: 0.2222, Specificity: 0.9091, F1 Score: 0.3333
Epoch 2/100
---------------
Val Loss: 1.0017
Accuracy: 0.5750, Precision: 0.5556, Recall: 0.2778, Specificity: 0.8182, F1 Score: 0.3704
Epoch 3/100
---------------
Val Loss: 0.8954
Accuracy: 0.6500, Precision: 0.7000, Recall: 0.3889, Specificity: 0.8636, F1 Score: 0.5000
Epoch 4/100
---------------
Val Loss: 0.8516
Accuracy: 0.7500, Precision: 0.8333, Recall: 0.5556, Specificity: 0.9091, F1 Score: 0.6667
Epoch 5/100
---------------
Val Loss: 0.7861
Accuracy: 0.7250, Precision: 0.7692, Recall: 0.5556, Specificity: 0.8636, F1 Score: 0.6452
Epoch 6/100
---------------
Val Loss: 0.6898
Accuracy: 0.7500, Precision: 0.7857, Recall: 0.6111, Specificity: 0.8636, F1 Score: 0.6875
Epoch 7/100
---------------
Val Loss: 0.6009
Accuracy: 0.7500, Precision: 0.7857, Recall: 0.6111, Specificity: 0.8636, F1 Score: 0.6875
Epoch 8/100
---------------
Val Loss: 0.6029
Acc

In [28]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm
import numpy as np

# Specify the device to run the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Model Initialization and Load Weights
best_model = timm.create_model('convnextv2_base', pretrained=False)
best_model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(best_model.num_features, 2)
)

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/ConvnextV2_samping.pth'
best_model.load_state_dict(torch.load(model_path, map_location=device))
best_model.to(device)
best_model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN for each class, with Abnormal first
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(best_model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_32892\4255245792.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(model_path, map_l

Confusion Matrix:
[[21  3]
 [ 1 18]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9545    0.8750    0.9130        24
      Normal     0.8571    0.9474    0.9000        19

    accuracy                         0.9070        43
   macro avg     0.9058    0.9112    0.9065        43
weighted avg     0.9115    0.9070    0.9073        43

ROC-AUC Score: 0.9112

Class: Abnormal
True Positives (TP): 21
False Positives (FP): 3
False Negatives (FN): 1
True Negatives (TN): 18

Class: Normal
True Positives (TP): 18
False Positives (FP): 1
False Negatives (FN): 3
True Negatives (TN): 21

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9545
Recall: 0.8750
F1-Score: 0.9130
Support: 24.0

Normal:
Precision: 0.8571
Recall: 0.9474
F1-Score: 0.9000
Support: 19.0



# Resnet50

In [29]:
from torchvision import models

In [30]:
# 3. Model ResNet50
model = models.resnet50(pretrained=True)

# Mengganti fully connected layer
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True


g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.fc.parameters(), 'lr': 0.0001},
    {'params': model.conv1.parameters(), 'lr': 0.00001},
    {'params': model.layer1.parameters(), 'lr': 0.00001},
    {'params': model.layer2.parameters(), 'lr': 0.00001},
    {'params': model.layer3.parameters(), 'lr': 0.00001},
    {'params': model.layer4.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

In [32]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [33]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    # Initialize history lists
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    # Early stopping setup
    early_stopping = EarlyStopping(patience=10)

    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * inputs.size(0)

            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_loss = running_train_loss / total_train
        train_accuracy = correct_train / total_train

        # Validation
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_loss = running_val_loss / total_val
        val_accuracy = correct_val / total_val

        # Append to history
        train_loss_history.append(train_loss)
        val_loss_history.append(val_loss)
        train_accuracy_history.append(train_accuracy)
        val_accuracy_history.append(val_accuracy)

        # Print results
        print(f'Epoch {epoch+1}/{epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        # Update scheduler
        scheduler.step(val_loss)

        # Early stopping check
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    # Save the best model weights
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/ResNet50_samping.pth')
    model.load_state_dict(early_stopping.best_model_wts)

    return model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history

In [34]:
# 7. Model Evaluation on Test Data
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
    specificity = tn / (tn + fp)

    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')
    return accuracy, precision, recall, specificity, f1

In [35]:
# Running training
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [36]:
model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
Train Loss: 0.7332, Train Accuracy: 0.5000
Val Loss: 0.6960, Val Accuracy: 0.5250
Epoch 2/100
Train Loss: 0.6850, Train Accuracy: 0.5820
Val Loss: 0.6757, Val Accuracy: 0.6250
Epoch 3/100
Train Loss: 0.6506, Train Accuracy: 0.5820
Val Loss: 0.6547, Val Accuracy: 0.6500
Epoch 4/100
Train Loss: 0.6521, Train Accuracy: 0.6393
Val Loss: 0.6355, Val Accuracy: 0.6750
Epoch 5/100
Train Loss: 0.6450, Train Accuracy: 0.6311
Val Loss: 0.6160, Val Accuracy: 0.6500
Epoch 6/100
Train Loss: 0.5956, Train Accuracy: 0.7213
Val Loss: 0.6012, Val Accuracy: 0.6750
Epoch 7/100
Train Loss: 0.6060, Train Accuracy: 0.6721
Val Loss: 0.5846, Val Accuracy: 0.6750
Epoch 8/100
Train Loss: 0.5926, Train Accuracy: 0.6967
Val Loss: 0.5727, Val Accuracy: 0.6750
Epoch 9/100
Train Loss: 0.5779, Train Accuracy: 0.7049
Val Loss: 0.5614, Val Accuracy: 0.6750
Epoch 10/100
Train Loss: 0.5802, Train Accuracy: 0.6721
Val Loss: 0.5482, Val Accuracy: 0.7000
Epoch 11/100
Train Loss: 0.5280, Train Accuracy: 0.7623
Val

In [37]:
# Evaluation on test data
evaluate_model(model, test_loader)

Accuracy: 0.8605, Precision: 0.7826, Recall: 0.9474, Specificity: 0.7917, F1 Score: 0.8571


(0.8604651162790697,
 0.782608695652174,
 0.9473684210526315,
 0.7916666666666666,
 0.8571428571428571)

In [38]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

# device to run the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Load ResNet50 Model
model = models.resnet50(pretrained=False)

# fully connected layer
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# 2. Load saved model weights
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/ResNet50_samping.pth'
state_dict = torch.load(model_path, map_location=device, weights_only=True)

new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('fc.') and not (k.endswith('.1.weight') or k.endswith('.1.bias')):
        new_state_dict[k.replace('fc.', 'fc.1.')] = v
    else:
        new_state_dict[k] = v

# Load the adjusted state_dict into the model
model.load_state_dict(new_state_dict, strict=False)

# Move model to device
model.to(device)
model.eval()


# 3. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 4. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 5. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN for each class, with Abnormal first
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 6. Model Evaluation on Test Data
evaluate_model(model, test_loader)

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Confusion Matrix:
[[19  5]
 [ 1 18]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9500    0.7917    0.8636        24
      Normal     0.7826    0.9474    0.8571        19

    accuracy                         0.8605        43
   macro avg     0.8663    0.8695    0.8604        43
weighted avg     0.8760    0.8605    0.8608        43

ROC-AUC Score: 0.8695

Class: Abnormal
True Positives (TP): 19
False Positives (FP): 5
False Negatives (FN): 1
True Negatives (TN): 18

Class: Normal
True Positives (TP): 18
False Positives (FP): 1
False Negatives (FN): 5
True Negatives (TN): 19

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9500
Recall: 0.7917
F1-Score: 0.8636
Support: 24.0

Normal:
Precision: 0.7826
Recall: 0.9474
F1-Score: 0.8571
Support: 19.0



# ResnetRS

In [7]:
# 3. Model ResNetRS
model = timm.create_model('resnetrs50', pretrained=True)

# model head for two-class classification
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

In [8]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

criterion = nn.CrossEntropyLoss(weight=class_weights)

In [9]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [10]:
# 6. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/ResnetRS_samping.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model

In [11]:
# 7. Model Evaluation on Test Data
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

In [12]:
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [13]:
# 8. Train and Evaluate the Model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.5798
Accuracy: 0.6000, Precision: 0.5294, Recall: 1.0000, F1 Score: 0.6923
Epoch 2/100
---------------
Val Loss: 0.5241
Accuracy: 0.7000, Precision: 0.6000, Recall: 1.0000, F1 Score: 0.7500
Epoch 3/100
---------------
Val Loss: 0.4986
Accuracy: 0.7750, Precision: 0.6800, Recall: 0.9444, F1 Score: 0.7907
Epoch 4/100
---------------
Val Loss: 0.4626
Accuracy: 0.8000, Precision: 0.7273, Recall: 0.8889, F1 Score: 0.8000
Epoch 5/100
---------------
Val Loss: 0.4269
Accuracy: 0.8750, Precision: 0.8824, Recall: 0.8333, F1 Score: 0.8571
Epoch 6/100
---------------
Val Loss: 0.4022
Accuracy: 0.8750, Precision: 0.9333, Recall: 0.7778, F1 Score: 0.8485
Epoch 7/100
---------------
Val Loss: 0.4129
Accuracy: 0.8750, Precision: 0.9333, Recall: 0.7778, F1 Score: 0.8485
Epoch 8/100
---------------
Val Loss: 0.4407
Accuracy: 0.8500, Precision: 0.9286, Recall: 0.7222, F1 Score: 0.8125
Epoch 9/100
---------------
Val Loss: 0.5606
Accuracy: 0.8000, Precision: 0.9167

In [14]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

# Specify the device to run the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 3. Model ResNetRS
model = timm.create_model('resnetrs50', pretrained=True)

model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

# 2. Load saved model weights
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/ResnetRS_samping.pth'
state_dict = torch.load(model_path, map_location=device, weights_only=True)

new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('fc.') and not (k.endswith('.1.weight') or k.endswith('.1.bias')):
        new_state_dict[k.replace('fc.', 'fc.1.')] = v
    else:
        new_state_dict[k] = v

# Load the adjusted state_dict into the model
model.load_state_dict(new_state_dict, strict=False)

# Move model to device
model.to(device)
model.eval()


# 3. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 4. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 5. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN for each class, with Abnormal first
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 6. Model Evaluation on Test Data
evaluate_model(model, test_loader)

Confusion Matrix:
[[22  2]
 [ 4 15]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.8462    0.9167    0.8800        24
      Normal     0.8824    0.7895    0.8333        19

    accuracy                         0.8605        43
   macro avg     0.8643    0.8531    0.8567        43
weighted avg     0.8621    0.8605    0.8594        43

ROC-AUC Score: 0.8531

Class: Abnormal
True Positives (TP): 22
False Positives (FP): 2
False Negatives (FN): 4
True Negatives (TN): 15

Class: Normal
True Positives (TP): 15
False Positives (FP): 4
False Negatives (FN): 2
True Negatives (TN): 22

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.8462
Recall: 0.9167
F1-Score: 0.8800
Support: 24.0

Normal:
Precision: 0.8824
Recall: 0.7895
F1-Score: 0.8333
Support: 19.0



# DeiT

In [48]:
# Import DeiT Model from timm
model = timm.create_model('deit_base_patch16_224', pretrained=True)

# head for output to two classes
model.head = nn.Sequential(
    nn.Linear(model.num_features, 2)
)

for param in model.parameters():
    param.requires_grad = True

In [49]:
# Using device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)

In [50]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [51]:
# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        # Saving the best model weights
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [52]:
# Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation phase
        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculating evaluation metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        # Update scheduler dan cek early stopping
        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Load the best weight to the model
    model.load_state_dict(early_stopping.best_model_wts)

    # Save the best model to file
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/DeiT_samping.pth')
    print("Model terbaik telah disimpan ke path yang ditentukan.")

    return model


In [53]:
# Training model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Val Loss: 0.4450
Accuracy: 0.6250, Precision: 0.5484, Recall: 0.9444, Specificity: 0.3636, F1 Score: 0.6939
Epoch 2/100
---------------
Val Loss: 0.3281
Accuracy: 0.7250, Precision: 0.6296, Recall: 0.9444, Specificity: 0.5455, F1 Score: 0.7556
Epoch 3/100
---------------
Val Loss: 0.3010
Accuracy: 0.7750, Precision: 0.6957, Recall: 0.8889, Specificity: 0.6818, F1 Score: 0.7805
Epoch 4/100
---------------
Val Loss: 0.3119
Accuracy: 0.8500, Precision: 0.8333, Recall: 0.8333, Specificity: 0.8636, F1 Score: 0.8333
Epoch 5/100
---------------
Val Loss: 0.3171
Accuracy: 0.8500, Precision: 0.8333, Recall: 0.8333, Specificity: 0.8636, F1 Score: 0.8333
Epoch 6/100
---------------
Val Loss: 0.3552
Accuracy: 0.8000, Precision: 0.7083, Recall: 0.9444, Specificity: 0.6818, F1 Score: 0.8095
Epoch 7/100
---------------
Val Loss: 0.3311
Accuracy: 0.8500, Precision: 0.8000, Recall: 0.8889, Specificity: 0.8182, F1 Score: 0.8421
Epoch 8/100
---------------
Val Loss: 0.4257
Accuracy: 0.7750, Precision: 0.

In [54]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# Specify the device to run the model
best_model = timm.create_model('deit_base_patch16_224', pretrained=True)

# head for output to two classes
best_model.head = nn.Sequential(
    nn.Linear(best_model.num_features, 2)
)

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/DeiT_samping.pth'
best_model.load_state_dict(torch.load(model_path, map_location=device))
best_model.to(device)
best_model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(best_model, test_loader)

C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_32892\2467741027.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(model_path, map_l

Confusion Matrix:
[[16  8]
 [ 1 18]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9412    0.6667    0.7805        24
      Normal     0.6923    0.9474    0.8000        19

    accuracy                         0.7907        43
   macro avg     0.8167    0.8070    0.7902        43
weighted avg     0.8312    0.7907    0.7891        43

ROC-AUC Score: 0.8070

Specificity : 0.6667

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9412
Recall: 0.6667
F1-Score: 0.7805
Support: 24.0

Normal:
Precision: 0.6923
Recall: 0.9474
F1-Score: 0.8000
Support: 19.0



# MFormer

In [55]:
# 3. Model MFormer
# Using timm to load MFormer
model = timm.create_model('mixer_b16_224', pretrained=True)

# head for two class classification
model.head = nn.Sequential(
    nn.LayerNorm(model.num_features),
    nn.Linear(model.num_features, 2)
)

# Enable gradients untuk fine-tuning semua layer
for param in model.parameters():
    param.requires_grad = True

In [56]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Accessing stem parameters instead of patch_embed
optimizer = optim.AdamW([
    {'params': model.head.parameters(), 'lr': 0.0001},
    {'params': model.stem.parameters(), 'lr': 0.00001},
    {'params': model.blocks.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

# learning rate scheduler with reduction when loss is stagnant
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [57]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [58]:
# 6. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/MFormer_samping.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [59]:
# 8. Train model and save the best weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)


Epoch 1/100
---------------


Val Loss: 1.0364
Accuracy: 0.5500, Precision: 1.0000, Recall: 0.0000, Specificity: 1.0000, F1 Score: 0.0000
Epoch 2/100
---------------
Val Loss: 1.0364
Accuracy: 0.5500, Precision: 1.0000, Recall: 0.0000, Specificity: 1.0000, F1 Score: 0.0000
Epoch 3/100
---------------
Val Loss: 1.0233
Accuracy: 0.5500, Precision: 1.0000, Recall: 0.0000, Specificity: 1.0000, F1 Score: 0.0000
Epoch 4/100
---------------
Val Loss: 0.9049
Accuracy: 0.5750, Precision: 1.0000, Recall: 0.0556, Specificity: 1.0000, F1 Score: 0.1053
Epoch 5/100
---------------
Val Loss: 0.8110
Accuracy: 0.6000, Precision: 1.0000, Recall: 0.1111, Specificity: 1.0000, F1 Score: 0.2000
Epoch 6/100
---------------
Val Loss: 0.6798
Accuracy: 0.6500, Precision: 0.8333, Recall: 0.2778, Specificity: 0.9545, F1 Score: 0.4167
Epoch 7/100
---------------
Val Loss: 0.6144
Accuracy: 0.7250, Precision: 0.8889, Recall: 0.4444, Specificity: 0.9545, F1 Score: 0.5926
Epoch 8/100
---------------
Val Loss: 0.5779
Accuracy: 0.7250, Precision: 0.

In [60]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 1. Model MFormer
model = timm.create_model('mixer_b16_224', pretrained=True)

# head for two class classification
model.head = nn.Sequential(
    nn.LayerNorm(model.num_features),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/MFormer_samping.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Evaluasi Model pada Data Uji
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_32892\1076153216.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[22  2]
 [ 3 16]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.8800    0.9167    0.8980        24
      Normal     0.8889    0.8421    0.8649        19

    accuracy                         0.8837        43
   macro avg     0.8844    0.8794    0.8814        43
weighted avg     0.8839    0.8837    0.8833        43

ROC-AUC Score: 0.8794

Specificity : 0.9167

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.8800
Recall: 0.9167
F1-Score: 0.8980
Support: 24.0

Normal:
Precision: 0.8889
Recall: 0.8421
F1-Score: 0.8649
Support: 19.0



# Convnext

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

In [62]:
# 1. ConvNeXt Model
model = timm.create_model('convnext_base', pretrained=True)

in_features = model.get_classifier().in_features
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 2)
)

# Move model to device
model = model.to(device)

In [63]:
# 2. Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.head.parameters(), 'lr': 0.0001},
    {'params': model.stem.parameters(), 'lr': 0.00001},
    {'params': model.stages.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)


In [64]:
# 3. Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

In [65]:
# 4. Early Stopping Implementation
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [66]:
# 5. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50):
    early_stopping = EarlyStopping(patience=10)
    train_loss_history, val_loss_history = [], []
    train_accuracy_history, val_accuracy_history = [], []

    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        print('-' * 15)

        # Training Phase
        model.train()
        running_loss = 0.0
        all_preds, all_labels = [], []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        train_loss = running_loss / len(train_loader)
        train_accuracy = accuracy_score(all_labels, all_preds)
        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)

        # Validation Phase
        model.eval()
        val_loss = 0.0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(all_labels, all_preds)
        val_loss_history.append(val_loss)
        val_accuracy_history.append(val_accuracy)

        # Scheduler Step
        scheduler.step(val_loss)

        # Metrics Logging
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        # Early Stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    # Save the best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/Convnext_samping.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model

In [67]:
# 7. Training the Model
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Train Loss: 0.7717, Train Accuracy: 0.4836
Val Loss: 0.6956, Val Accuracy: 0.5750
Epoch 2/100
---------------
Train Loss: 0.6944, Train Accuracy: 0.5902
Val Loss: 0.6745, Val Accuracy: 0.7250
Epoch 3/100
---------------
Train Loss: 0.5903, Train Accuracy: 0.6803
Val Loss: 0.6572, Val Accuracy: 0.7500
Epoch 4/100
---------------
Train Loss: 0.5270, Train Accuracy: 0.7541
Val Loss: 0.6526, Val Accuracy: 0.7500
Epoch 5/100
---------------
Train Loss: 0.4862, Train Accuracy: 0.7377
Val Loss: 0.6631, Val Accuracy: 0.7500
Epoch 6/100
---------------
Train Loss: 0.4399, Train Accuracy: 0.8033
Val Loss: 0.6486, Val Accuracy: 0.7750
Epoch 7/100
---------------
Train Loss: 0.4472, Train Accuracy: 0.8115
Val Loss: 0.6554, Val Accuracy: 0.7750
Epoch 8/100
---------------
Train Loss: 0.4064, Train Accuracy: 0.8525
Val Loss: 0.5924, Val Accuracy: 0.7500
Epoch 9/100
---------------
Train Loss: 0.4204, Train Accuracy: 0.7951
Val Loss: 0.5086, Val Accuracy: 0.7750
Epoch 10/100
---------------
Train Los

In [68]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

model = timm.create_model('convnext_base', pretrained=True)

in_features = model.get_classifier().in_features
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 2)
)

# Move model to device
model = model.to(device)

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/Convnext_samping.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_32892\3935257498.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[19  5]
 [ 1 18]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9500    0.7917    0.8636        24
      Normal     0.7826    0.9474    0.8571        19

    accuracy                         0.8605        43
   macro avg     0.8663    0.8695    0.8604        43
weighted avg     0.8760    0.8605    0.8608        43

ROC-AUC Score: 0.8695

Specificity : 0.7917

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9500
Recall: 0.7917
F1-Score: 0.8636
Support: 24.0

Normal:
Precision: 0.7826
Recall: 0.9474
F1-Score: 0.8571
Support: 19.0



# RSMamba

In [69]:
# 3. Model RSMamba
model = timm.create_model('resnet50', pretrained=True)

# Head modification for 2 class output
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning
for param in model.parameters():
    param.requires_grad = True

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mahasiswa\.cache\huggingface\hub\models--timm--resnet50.a1_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [70]:
# Optimizer with different learning rates for specific layers
optimizer = optim.AdamW([
    {'params': model.fc.parameters(), 'lr': 0.0001},
    {'params': [param for name, param in model.named_parameters() if 'fc' not in name], 'lr': 0.00001}
], weight_decay=5e-4)

In [71]:
# 5. Callback Early Stopping
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [72]:
# 6. Training loop dengan Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100, class_weights=None): # Add class_weights as an argument
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Check if class_weights is provided, and move it to the device if so
    if class_weights is not None:
        class_weights = class_weights.to(device)

    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/RSMamba_samping.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [73]:
# 8. Train model and save the best weight
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.7200
Accuracy: 0.5500, Precision: 0.5000, Recall: 0.0556, Specificity: 0.9545, F1 Score: 0.1000
Epoch 2/100
---------------
Val Loss: 0.7084
Accuracy: 0.5250, Precision: 0.4286, Recall: 0.1667, Specificity: 0.8182, F1 Score: 0.2400
Epoch 3/100
---------------
Val Loss: 0.7007
Accuracy: 0.5250, Precision: 0.4615, Recall: 0.3333, Specificity: 0.6818, F1 Score: 0.3871
Epoch 4/100
---------------
Val Loss: 0.6947
Accuracy: 0.5250, Precision: 0.4706, Recall: 0.4444, Specificity: 0.5909, F1 Score: 0.4571
Epoch 5/100
---------------
Val Loss: 0.6922
Accuracy: 0.4750, Precision: 0.4348, Recall: 0.5556, Specificity: 0.4091, F1 Score: 0.4878
Epoch 6/100
---------------
Val Loss: 0.6930
Accuracy: 0.4250, Precision: 0.4194, Recall: 0.7222, Specificity: 0.1818, F1 Score: 0.5306
Epoch 7/100
---------------
Val Loss: 0.6924
Accuracy: 0.3750, Precision: 0.3939, Recall: 0.7222, Specificity: 0.0909, F1 Score: 0.5098
Epoch 8/100
---------------
Val Loss: 0.6927
Acc

In [74]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model RSMamba
model = timm.create_model('resnet50', pretrained=True)

# Head modification for 2 class output
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/RSMamba_samping.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_32892\1867193677.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[16  8]
 [ 1 18]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9412    0.6667    0.7805        24
      Normal     0.6923    0.9474    0.8000        19

    accuracy                         0.7907        43
   macro avg     0.8167    0.8070    0.7902        43
weighted avg     0.8312    0.7907    0.7891        43

ROC-AUC Score: 0.8070

Specificity : 0.6667

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9412
Recall: 0.6667
F1-Score: 0.7805
Support: 24.0

Normal:
Precision: 0.6923
Recall: 0.9474
F1-Score: 0.8000
Support: 19.0



# Alexnet

In [75]:
# 3. Model AlexNet
from torchvision.models import alexnet

# Initialize AlexNet pretrained model
model = alexnet(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\Mahasiswa/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:21<00:00, 11.4MB/s] 


In [76]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer setup
optimizer = optim.AdamW([
    {'params': model.features.parameters(), 'lr': 0.00001},
    {'params': model.classifier.parameters(), 'lr': 0.0001}
], weight_decay=5e-4)

# Scheduler setup
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [77]:
# 5. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        # Training phase
        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        train_accuracy = accuracy_score(all_labels, all_preds)
        train_loss = running_loss / len(train_loader)
        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)

        # Validation phase
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(all_labels, all_preds)
        val_loss = val_loss / len(val_loader)
        val_loss_history.append(val_loss)
        val_accuracy_history.append(val_accuracy)

        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        scheduler.step(val_loss)
        early_stopping(val_loss, model)

        if early_stopping.early_stop:
            print('Early stopping triggered.')
            break

    # Save the best model weights
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/AlexNet_samping.pth')
    model.load_state_dict(early_stopping.best_model_wts)

    return model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history


In [78]:
# 6. Train the AlexNet model
criterion = nn.CrossEntropyLoss(weight=class_weights)
trained_model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100
)

Epoch 1/100
---------------


Train Loss: 0.6648, Train Accuracy: 0.5984
Val Loss: 0.6924, Val Accuracy: 0.6750
Epoch 2/100
---------------
Train Loss: 0.5693, Train Accuracy: 0.7049
Val Loss: 0.5492, Val Accuracy: 0.8500
Epoch 3/100
---------------
Train Loss: 0.5098, Train Accuracy: 0.7623
Val Loss: 0.6125, Val Accuracy: 0.8250
Epoch 4/100
---------------
Train Loss: 0.4355, Train Accuracy: 0.8279
Val Loss: 0.5054, Val Accuracy: 0.8500
Epoch 5/100
---------------
Train Loss: 0.4744, Train Accuracy: 0.8279
Val Loss: 0.4854, Val Accuracy: 0.8500
Epoch 6/100
---------------
Train Loss: 0.5207, Train Accuracy: 0.7951
Val Loss: 0.4083, Val Accuracy: 0.8500
Epoch 7/100
---------------
Train Loss: 0.4562, Train Accuracy: 0.7541
Val Loss: 0.3613, Val Accuracy: 0.8750
Epoch 8/100
---------------
Train Loss: 0.4157, Train Accuracy: 0.8115
Val Loss: 0.2614, Val Accuracy: 0.8500
Epoch 9/100
---------------
Train Loss: 0.3339, Train Accuracy: 0.8607
Val Loss: 0.2563, Val Accuracy: 0.9250
Epoch 10/100
---------------
Train Los

In [79]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model AlexNet
from torchvision.models import alexnet

# Initialize AlexNet pretrained model
model = alexnet(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/AlexNet_samping.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_32892\621737140.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob

Confusion Matrix:
[[20  4]
 [ 2 17]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9091    0.8333    0.8696        24
      Normal     0.8095    0.8947    0.8500        19

    accuracy                         0.8605        43
   macro avg     0.8593    0.8640    0.8598        43
weighted avg     0.8651    0.8605    0.8609        43

ROC-AUC Score: 0.8640

Specificity : 0.8333

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9091
Recall: 0.8333
F1-Score: 0.8696
Support: 24.0

Normal:
Precision: 0.8095
Recall: 0.8947
F1-Score: 0.8500
Support: 19.0



# EfficientNet V2

In [80]:
# 3. Model EfficientNetV2
model = timm.create_model('tf_efficientnetv2_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mahasiswa\.cache\huggingface\hub\models--timm--tf_efficientnetv2_b0.in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [81]:
# 4. Loss function, optimizer, dan scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer with different learning rates for classifier and backbone
optimizer = optim.AdamW([
    {'params': model.classifier.parameters(), 'lr': 0.0001},
    {'params': model.conv_stem.parameters(), 'lr': 0.00001},
    {'params': model.blocks.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

In [82]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Loss function dengan class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [83]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [84]:
# 6. Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/EfficientNetV2_samping.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [85]:
# 8. Train model and save the best weights
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Val Loss: 0.6843
Accuracy: 0.5750, Precision: 0.5294, Recall: 0.5000, Specificity: 0.6364, F1 Score: 0.5143
Epoch 2/100
---------------
Val Loss: 0.6724
Accuracy: 0.6000, Precision: 0.5625, Recall: 0.5000, Specificity: 0.6818, F1 Score: 0.5294
Epoch 3/100
---------------
Val Loss: 0.6684
Accuracy: 0.7000, Precision: 0.6667, Recall: 0.6667, Specificity: 0.7273, F1 Score: 0.6667
Epoch 4/100
---------------
Val Loss: 0.6698
Accuracy: 0.7250, Precision: 0.7059, Recall: 0.6667, Specificity: 0.7727, F1 Score: 0.6857
Epoch 5/100
---------------
Val Loss: 0.6714
Accuracy: 0.7250, Precision: 0.7059, Recall: 0.6667, Specificity: 0.7727, F1 Score: 0.6857
Epoch 6/100
---------------
Val Loss: 0.6720
Accuracy: 0.6750, Precision: 0.6471, Recall: 0.6111, Specificity: 0.7273, F1 Score: 0.6286
Epoch 7/100
---------------
Val Loss: 0.6713
Accuracy: 0.6750, Precision: 0.6471, Recall: 0.6111, Specificity: 0.7273, F1 Score: 0.6286
Epoch 8/100
---------------
Val Loss: 0.6748
Accuracy: 0.7000, Precision: 0.

In [86]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model EfficientNetV2
model = timm.create_model('tf_efficientnetv2_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/EfficientNetV2_samping.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_32892\2816639316.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[20  4]
 [10  9]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.6667    0.8333    0.7407        24
      Normal     0.6923    0.4737    0.5625        19

    accuracy                         0.6744        43
   macro avg     0.6795    0.6535    0.6516        43
weighted avg     0.6780    0.6744    0.6620        43

ROC-AUC Score: 0.6535

Specificity : 0.8333

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.6667
Recall: 0.8333
F1-Score: 0.7407
Support: 24.0

Normal:
Precision: 0.6923
Recall: 0.4737
F1-Score: 0.5625
Support: 19.0



# EfficientNetBO

In [15]:
# 3. Model EfficientNet-B0
model = timm.create_model('efficientnet_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.blocks.parameters(), 'lr': 0.00001},
    {'params': model.classifier.parameters(), 'lr': 0.0001}
], weight_decay=5e-4)

In [17]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Loss function dengan class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [18]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [19]:
# 6. Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/EfficientNetBO_samping.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [22]:
# 8. Train model and save the best weights
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.4729
Accuracy: 0.7500, Precision: 0.7500, Recall: 0.6667, Specificity: 0.8182, F1 Score: 0.7059
Epoch 2/100
---------------
Val Loss: 0.4740
Accuracy: 0.7500, Precision: 0.7500, Recall: 0.6667, Specificity: 0.8182, F1 Score: 0.7059
Epoch 3/100
---------------
Val Loss: 0.4659
Accuracy: 0.7500, Precision: 0.7500, Recall: 0.6667, Specificity: 0.8182, F1 Score: 0.7059
Epoch 4/100
---------------
Val Loss: 0.4704
Accuracy: 0.7500, Precision: 0.7500, Recall: 0.6667, Specificity: 0.8182, F1 Score: 0.7059
Epoch 5/100
---------------
Val Loss: 0.4784
Accuracy: 0.7500, Precision: 0.7500, Recall: 0.6667, Specificity: 0.8182, F1 Score: 0.7059
Epoch 6/100
---------------
Val Loss: 0.4839
Accuracy: 0.7500, Precision: 0.7500, Recall: 0.6667, Specificity: 0.8182, F1 Score: 0.7059
Epoch 7/100
---------------
Val Loss: 0.4818
Accuracy: 0.7250, Precision: 0.7333, Recall: 0.6111, Specificity: 0.8182, F1 Score: 0.6667
Epoch 8/100
---------------
Val Loss: 0.4835
Acc

In [23]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model EfficientNet-B0
model = timm.create_model('efficientnet_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/modelSamping/EfficientNetBO_samping.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Samping/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_15732\1899257510.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[22  2]
 [ 3 16]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.8800    0.9167    0.8980        24
      Normal     0.8889    0.8421    0.8649        19

    accuracy                         0.8837        43
   macro avg     0.8844    0.8794    0.8814        43
weighted avg     0.8839    0.8837    0.8833        43

ROC-AUC Score: 0.8794

Specificity : 0.9167

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.8800
Recall: 0.9167
F1-Score: 0.8980
Support: 24.0

Normal:
Precision: 0.8889
Recall: 0.8421
F1-Score: 0.8649
Support: 19.0

